# Install necessary libs, if the computer don't have these (uncomment necessary lib)

In [1]:
# !pip install paramiko
# !pip install mysql-connector-python
# !pip install sshtunnel
# !pip install numpy
# !pip install pandas

# Load Libs

In [2]:
import paramiko
import mysql.connector
import sshtunnel
import numpy as np
import pandas as pd
import timeit

***
* Check for NaN function

In [3]:
def check_nan(df):
    missing_val_count_by_column = (df.isna().sum())
    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

* chronometer

In [4]:
def check_time(start, end):
    seg = end - start
    dias = seg // 86400
    Seg_Restantes_Após_Dias = seg % 86400
    horas = Seg_Restantes_Após_Dias // 3600         # Divisão inteira
    Seg_Restantes_Apos_Hora = seg % 3600            # Resto da divisão por 3600
    minutos = Seg_Restantes_Apos_Hora // 60         # Divisão inteira dos segundos restantes
    Seg_Restantes_Apos_Minutos = seg % 60
    #print(minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')
    print(dias,'dias', horas,'horas', minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')

***
* MySQL connector

In [22]:
connection = mysql.connector.MySQLConnection(user="root",
                                             password="",
                                             host='127.0.0.1',
                                             database='coldchain',
                                             port=3306)

***
# Array to Dataframe

**datapoint**' - Cada tupla da tabela representa uma (i) leitura de eHook/tag OU (ii) uma localização aferida pelo smartphone. Caso (i), a coluna `session_id` será diferente de nulo, caso (ii) `session_id` será nulo, indicando que não a tupla não tem relação com tag, apenas com o escanner/celular.

features importantes:

'id' = identificador;

'datetime' = horario da leitura

'latitude e longitude' = localização;

'session_id' = se é ehook ou smartphone 

               ehook / tag != NULL \
               smartfone == NULL (se comporta como SLS);
             
'appinstance_id' = tabela appinstance - origem do dado


In [6]:
df_datapoint = pd.read_sql_query('''SELECT id, datetime, latitude, longitude, geo_accuracy, session_id, appinstance_id
                                    FROM coldchain.datapoint
                                    WHERE (session_id IS null)
                                    LIMIT 10;''', connection)
df_datapoint

,id,datetime,latitude,longitude,geo_accuracy,session_id,appinstance_id
0,9021239,2019-02-04 17:01:33,-22.042549,-47.892273,42.000,None,15
1,9021414,2019-02-04 17:02:09,-22.042549,-47.892273,42.000,None,15
2,9022165,2019-02-04 17:04:14,-22.042549,-47.892273,42.000,None,15
3,9022259,2019-02-04 17:05:26,-22.042549,-47.892273,42.000,None,15
4,9022279,2019-02-04 17:15:25,-22.042549,-47.892273,42.000,None,15
5,9028113,2019-02-04 17:27:07,-22.007919,-47.888231,16.091,None,15
6,9028128,2019-02-04 17:57:11,-22.007919,-47.888231,16.091,None,15
7,9028129,2019-02-04 18:09:04,-22.007962,-47.888253,16.291,None,19
8,9028130,2019-02-04 18:09:04,-22.007962,-47.888253,16.291,None,19
9,9029811,2019-02-04 18:10:50,-22.007991,-47.888249,16.416,None,19


=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* **session** - Um tag/eHook/#tag pode ter múltiplas sessões. Cada sessão possui múltiplas leituras atreladas. É apenas uma organização das leituras em um dado contexto. No logger do coldchain é uma viagem que o logger faz, no eHook está em desuso, apesar da estrutura ser populada

'id' = identificador;



In [7]:
df_session = pd.read_sql_query('''SELECT id, identifier, count, last_datapoint_time, tag_id
                                  FROM coldchain.session 
                                  WHERE temperature_max IS NULL 
                                  LIMIT 10;''', connection)
df_session

,id,identifier,count,last_datapoint_time,tag_id
0,99,1,1478,2018-10-05 23:57:20.489,17
1,100,2,1158,2018-10-08 21:37:31.738,16
2,101,2,1301,2018-10-06 23:58:41.098,17
3,102,3,1379,2018-10-08 00:11:05.173,17
4,103,4,1610,2018-10-09 01:12:58.375,17
5,104,1001,4885,2018-11-07 18:57:45.731,17
6,105,1002,7213,2018-10-24 19:58:59.653,16
7,106,2001,43472,2018-12-13 12:48:05.912,20
8,107,2002,1006,2018-11-05 14:40:41.739,18
9,108,2003,17143,2018-11-13 19:48:18.338,19


* **tag** - Um tag pode ser um logger ColdCshapen, tag 915MHz PA, eHook, etc. Cada eHook também possui uma tupla na tabela **tag_ehook**, que permite determinar quais tuplas da tabela tag são referentes a eHooks

In [8]:
df_tag = pd.read_sql_query("""SELECT id, tid, epc, alias, last_datapoint_time, company_id, owner_id, place_tag_id 
                              FROM coldchain.tag
                              ORDER BY id;""", connection)
df_tag

,id,tid,epc,alias,last_datapoint_time,company_id,owner_id,place_tag_id
0,1,C8,CAB0F06E00000000C8,TSC,2019-03-21 11:30:06.059,6,6,None
1,2,75BCD15,CAB0F06E000000001E13E58,None,2019-03-21 11:30:06.059,6,6,None
2,3,77A10,CAB0F06E000000005BCD77,None,2019-03-21 11:30:06.059,6,6,None
3,4,CAB020180327F083,CAB0F0830000000035A4E900,Q83,2019-03-21 11:30:06.059,6,6,None
4,5,CAB020180327F084,CAB0F0840000000035A4E901,Q84,2019-03-21 11:30:06.059,6,6,None
...,...,...,...,...,...,...,...,...
11100,11158,5243945,CAB0F06E0000000000500429,90FD9F500429,2020-04-09 11:12:48.086,3,3,None
11101,11159,5243946,CAB0F06E000000000050042A,90FD9F50042A,2020-04-09 11:12:48.086,3,3,None
11102,11160,5243948,CAB0F06E000000000050042C,90FD9F50042C,2020-04-09 11:12:48.086,3,3,None
11103,11161,5243949,CAB0F06E000000000050042D,90FD9F50042D,2020-04-09 11:12:48.086,3,3,None


* **tag_ehook**, que permite determinar quais tuplas da tabela tag são referentes a eHooks

In [9]:
df_tag_ehook = pd.read_sql_query("""SELECT id, timestamp_base, tag_id, tag_group_id, tag_use_id
                                   FROM coldchain.tag_ehook    
                                   ORDER BY id
                                   LIMIT 10;""", connection)
df_tag_ehook

,id,timestamp_base,tag_id,tag_group_id,tag_use_id
0,1,2018-09-26 10:00:00.000,16,1,None
1,2,2018-09-26 12:00:00.000,17,2,None
2,3,2018-10-15 20:01:38.461,18,3,None
3,4,2018-10-15 20:01:38.475,19,4,None
4,5,2018-10-15 20:01:38.491,20,5,None
5,6,2018-10-15 20:01:38.498,21,6,None
6,7,2018-10-15 20:01:38.505,22,7,None
7,8,2018-10-15 20:01:38.512,23,8,None
8,9,2018-10-15 20:01:38.519,24,9,None
9,10,2018-10-15 20:01:38.526,25,10,None


* Separar e extrair leituras dos eHooks por KM, por dia e período do dia

'session_id' = se é ehook ou smartphone 

               ehook / tag != NULL

In [10]:
# print(df_ehook_km.to_string())

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* **appinstance** - representa a instalação de um App, integrado à lib Android/iOS eTracker, em um aparelho celular. Um aparelho pode ter múltiplos aplicativos que implementam a eTracker. Cada tupla da tabela datapoint necessariamente tem um appinstance associado, que é de onde veio o dado

In [11]:
start=timeit.default_timer()
df_appinstance = pd.read_sql_query("""SELECT id, position, company_id, sector_id, otacfg_id, reporter_id
                                      FROM coldchain.appinstance
                                      LIMIT 10;""", connection)
end=timeit.default_timer()
check_time(start, end)

df_appinstance

0.0 dias 0.0 horas 0.0 minutos e 0.44 segundos


,id,position,company_id,sector_id,otacfg_id,reporter_id
0,1,None,2,None,1,1
1,2,None,2,None,1,5
2,3,None,2,None,1,6
3,4,None,2,None,1,8
4,5,None,2,None,1,10
5,6,None,2,None,1,11
6,7,None,2,None,1,12
7,8,None,2,None,1,13
8,9,None,2,None,1,14
9,10,None,2,None,1,15


* **reporter** - o aparelho celular ou SLS que é dispositivo dedicado de leitura de eHooks / #tag

In [12]:
df_reporter = pd.read_sql_query("""SELECT id, identifier, alias, user_agent, reporterdevice_id 
                                   FROM coldchain.reporter
                                   WHERE reporterdevice_id IS NOT NULL
                                   ORDER BY id
                                   LIMIT 10
                                   ;""", connection)
df_reporter

,id,identifier,alias,user_agent,reporterdevice_id
0,20,9d01e2da8dea25c9ead4f12b44ffab4b,Ability A5,okhttp/3.11.0,122
1,21,2c89c27f6a0e506fb4432e28e2dbf43a,motoe vb,okhttp/3.11.0,121
2,22,97a33112c7132f8f813b7f5dba5d68df,Renan Campoa,okhttp/3.11.0,70
3,23,ade9e897db990ccd5b83c89b5bd83ded,s5,okhttp/3.11.0,95
4,24,3224f43e39827ade823e71998879ec79,Galaxy s8,okhttp/3.11.0,146
5,25,bb9f12fc57e5507a87e9b9957a41bae6,VB Campinas Lenovo,None,25
6,26,652336809e5b8b4b82a94817f3a8a6fc,Danilo Samsung S8,None,72
7,27,4cb2aac0bbc4221eee1c3e5f9ae3ac3e,Rodrigo MotoG5,None,13
8,28,236c9a47408057004a054007638265a1,Renan Silva,None,99
9,29,05ee365508d469758be225aa516f8ed0,Victor Asus,None,118


* reporterdevice

In [13]:
df_reporterdevice = pd.read_sql_query("""SELECT id, devicemodel_id, deviceosversion_id, libversion_id
                                         FROM coldchain.reporterdevice    
                                         ORDER BY id
                                         LIMIT 10;""", connection)
df_reporterdevice

,id,devicemodel_id,deviceosversion_id,libversion_id
0,2,2,2,1
1,3,3,3,2
2,4,4,4,2
3,5,5,5,2
4,6,6,6,2
5,7,7,7,2
6,8,8,2,3
7,9,9,8,3
8,10,10,2,3
9,11,11,8,3


* deviceosversion

In [14]:
df_deviceosversion = pd.read_sql_query("""SELECT *
                                         FROM coldchain.deviceosversion
                                         ORDER BY id
                                         LIMIT 10;""", connection)
df_deviceosversion.drop(['create_by', 'create_time', 'update_by', 'update_time', 'active'], axis=1)

,id,release,sdk
0,1,8739,23136
1,2,8.0.0,26
2,3,16532,20344
3,4,8129,20139
4,5,9159,4989
5,6,16609,4436
6,7,30442,12424
7,8,7.1.1,25
8,9,6.0.1,23
9,10,8.1.0,27


* Separar e extrair leituras dos reporters por KM, por dia e período do dia

'session_id' = se é ehook ou smartphone 

               smartfone == NULL (se comporta como SLS);

* status

In [30]:
connection = mysql.connector.MySQLConnection(user="root",
                                             password="",
                                             host='127.0.0.1',
                                             database='coldchain',
                                             port=3306)

In [31]:
df_reporterdevice = pd.read_sql_query("""SELECT id, bluetooth, datapoint_id
                                         FROM coldchain.status
                                         WHERE bluetooth is not null
                                         ORDER BY id;""", connection)
df_reporterdevice


KeyboardInterrupt



=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* Close Connections

In [15]:
connection.close()

print('FINISH!')

FINISH!
